In [1]:
'''
tsv to xml

1.load
    추후 메모리문제 위해선 한줄씩 처리 필요
2.filter
    개별함수 만들어 조합하는식으로
3.export xml
    변수값에따라 xml생성, 저장
'''

'\ntsv to xml\n\n1.load\n    추후 메모리문제 위해선 한줄씩 처리 필요\n2.filter\n    개별함수 만들어 조합하는식으로\n3.export xml\n    변수값에따라 xml생성, 저장\n'

## Import

In [2]:
import pandas as pd
import os
import re
import csv
from urllib import parse

### Load data

In [3]:
# dataframe
df = pd.read_csv("./data/fb.tsv", sep="\t")

In [4]:
len(df)
df['custom_label_0'][:5]

0    http://m.maychic.co.kr/product/detail.html?pro...
1    http://m.maychic.co.kr/product/detail.html?pro...
2    http://m.maychic.co.kr/product/detail.html?pro...
3    http://m.maychic.co.kr/product/detail.html?pro...
4    http://m.maychic.co.kr/product/detail.html?pro...
Name: custom_label_0, dtype: object

In [5]:
# 수정시 메모리문제 위해선 한줄씩 처리해야함
# with open
with open('./data/fb.tsv', 'r', encoding='utf-8', newline='') as f:
    line = f.readlines()
#     print(line)    
    f.close()

### Filter Class

In [6]:
class FeedFilter():
    def __init__(self,text):
        self._text = text
        
    def setText(self, text):
        self._text = str(text)
        return self
    
    def getText(self):
        return self._text
        
    # 길이제한
    def limitTextLength(self, length):                
        self.setText( self.getText()[:length])
        return self
    
    # 단어제거
    def deleteText(self, stopWord) :        
        self.setText(self.getText().replace(stopWord,''))
        return self
    
    # html태그 제거
    def deleteHTMLTags(self) :
        self.setText( re.sub(r'<.*?>','',self.getText()) )
        return self
    
    # link http://시작, 기호 인코딩
    def fitURL(self):
        query = ''
        result = ''
        urlparse = parse.urlparse(self._text)
#         print(urlparse)

        # scheme 
        if urlparse.scheme=='':
            urlparse = urlparse._replace(scheme = 'http')

        if urlparse.query!='':
            query = parse.parse_qs(urlparse.query)
#             print(query)

            # 쿼리 인코딩
            query = '?'+parse.urlencode(query, doseq=True)
#             print(query)

        self.setText( urlparse.scheme+'://'+urlparse.netloc+urlparse.path + query )

        return self
    
    # ,제거 .소숫점 허용
    def fitNumber(self):        
        self.setText( self.getText().replace(',',''))
        return self
    
    # 문자열 제한
    def checkTextInList(self, limitList):
        if self.getText().lower() in limitList:
            return True
        else :
            return False        
    
    # 자리수 제한 숫자(문자열)
    def checkNumberLengthInList(self, lengthList):        
        if len(str(self.getText())) in lengthList: 
            return  True        
        else :             
            return False
        
    def limitTextAscii(self):
        result = ''
        for char in self.getText():
            if ord(char) < 127 :
                result = result + char
        self.setText( result )
        return self
        


In [7]:
# ascii만 포함
ff = FeedFilter('')

In [8]:
print( ff.setText('<title>232"323"232')
      .deleteText('"')
      .limitTextLength(10)      
      .getText() )

<title>232


In [56]:
print(ff.setText('a"b"c')
        .deleteText('"')
        .getText())

abc


In [57]:
# 문자나 숫자로만 시작


In [58]:
print( ff.setText('1234<title>5678</title>910')
        .deleteHTMLTags()
        .getText())

12345678910


In [13]:
# print( fitURL('www.exeam.org?examParam1=value1&examParam2=한글$$') )
print( ff.setText('http://maychic.co.kr/web/product/medium/202008/9bc7171c8a4681b000dcaea0b985c57a_dburl.jpg')
        .fitURL()
        .getText())

ParseResult(scheme='http', netloc='maychic.co.kr', path='/web/product/medium/202008/9bc7171c8a4681b000dcaea0b985c57a_dburl.jpg', params='', query='', fragment='')
http://maychic.co.kr/web/product/medium/202008/9bc7171c8a4681b000dcaea0b985c57a_dburl.jpg


In [14]:
print( ff.setText('2,000,000.34565')
        .fitNumber()
        .getText())

2000000.34565


In [16]:
print( ff.setText('bbba')
        .checkTextInList(['aaa','bbb','ccc']))

False


In [100]:
print( ff.setText('000222121')
          .checkNumberLengthInList([8,12,13,14]) )

False


In [158]:
temp = 'sdfs sdf12 331!@#!%^*<>< ><>>>★ 최저가 보장'
print(ff.setText(temp).limitTextAscii().getText())



sdfs sdf12 331!@#!%^*<>< ><>>>  


In [193]:
##
list(df.columns)

['id',
 'availability',
 'condition',
 'description',
 'image_link',
 'link',
 'custom_label_0',
 'title',
 'price',
 'sale_price',
 'gtin',
 'mpn',
 'brand',
 'product_type']

In [9]:
#test dataframe
tempdf = pd.DataFrame()

In [10]:
'''
문자 요구 사항: id는 ASCII 문자만 포함해야 하며 따옴표를 포함할 수 없습니다.

제한: 240

유형: String

예: abc123-black-7.5
'''
tempdf['id'] = df['id'].apply(
    lambda x : ff.setText(x)
                .limitTextAscii()
                .limitTextLength(240)
                .getText()
)
print(tempdf['id'][:3])

0    13
1    27
2    32
Name: id, dtype: object


In [11]:
'''
문자 요구 사항: availability는 다음 세가지 값 중 하나로 채워 져야합니다.

preorder
out of stock
in stock
제한: 25

유형: String

예: in stock
'''

def getAvailability(x):
    ff = FeedFilter('')
    if ff.setText(x).limitTextLength(25).checkTextInList(['preorder','out of stock','in stock']):
        return ff.getText()
    else :
        return False

tempdf['availability'] = df['availability'].apply(
    lambda x : getAvailability(x)
)
print(tempdf['availability'][:3])

0    in stock
1    in stock
2    in stock
Name: availability, dtype: object


In [12]:
'''
문자 요구 사항: 세 가지 값만 허용됩니다:

'new' [신제품]
'refurbished' [리퍼브제품]
'used' [중고품]
유형: String

예: new

'''
def getCondition(x):
    ff = FeedFilter('')
    if ff.setText(x).checkTextInList(['new','resurbished','used']) :        
        return ff.getText()
    else : 
        return False
    
tempdf['condition'] = df['condition'].apply(
    lambda x : getCondition(x)
)
print(tempdf['condition'][:3])

0    NEW
1    NEW
2    NEW
Name: condition, dtype: object


In [13]:
'''
문자 요구사항: description은 문자나 숫자로 시작해야 합니다. 
style, embed, object, anchor 태그를 포함하여 이 필드의 모든 HTML은 제거되어야 합니다.

제한: 2500 (영문 5000) 

유형: String

예: 일상적 사용에 우수함.
'''
tempdf['description'] = df['description'].apply(
    lambda x : ff.setText(x)
                .deleteHTMLTags()
                .limitTextLength(2500)
                .getText()
)
print(tempdf['description'][:3])

0      적립금 있음 | 배송비 2500 | 리뷰 : 58
1      적립금 있음 | 배송비 2500 | 리뷰 : 77
2    적립금 있음 | 배송비 2500 | 리뷰 : 1153
Name: description, dtype: object


In [14]:
'''
문자 요구 사항: image_link는 제품 이미지의 전체 URL 앞에 http://나 https://로 시작해야 합니다. 
모든 기호는 인코딩해야 합니다. 예. $은 %24로 교체해야 합니다.  

제한: 2000

유형: String

예: http://www.example.com/product/image/working-boots.png
'''
tempdf['image_link'] = df['image_link'].apply(
    lambda x : ff.setText(x)
                .fitURL()
                .limitTextLength(2000)
                .getText()
)
print( tempdf['image_link'][:3].values )

['http://maychic.co.kr/web/product/medium/202008/9bc7171c8a4681b000dcaea0b985c57a_dburl.jpg'
 'http://maychic.co.kr/web/product/medium/202008/278ac9d9aa775b974fb4c79c7738dd09_dburl.jpg'
 'http://maychic.co.kr/web/product/medium/201908/72c85a9fefac81d6906b73824909f62f_dburl.jpg']


In [15]:
'''
문자 요구 사항: link는 제품 상세 페이지의 전체 URL 앞에 프로토콜로 (http:// 또는 https://) 시작해야 합니다. 모든 기호는 인코딩해야 합니다. 예. $은 %24로 교체해야 합니다.

제한: 1024

유형: String

예: http://www.example.com/product/working-boots
'''
tempdf['link'] = df['link'].apply(
    lambda x : ff.setText(x)
                .fitURL()
                .limitTextLength(1024)
                .getText()
)
print(tempdf['link'][:3].values)

['http://maychic.co.kr/product/detail.html?product_no=13'
 'http://maychic.co.kr/product/detail.html?product_no=27'
 'http://maychic.co.kr/product/detail.html?product_no=32']


In [16]:
'''
문자 요구 사항: 사용자 지정 레이블 중 5개 레벨만 허용함 (0에서 4까지)

제한: 100(영문 100)

유형: String
'''
tempdf['custom_label_0'] = df['custom_label_0'].apply(
    lambda x : ff.setText(x).getText()
)
print(tempdf['custom_label_0'][:3].values)

['http://m.maychic.co.kr/product/detail.html?product_no=13'
 'http://m.maychic.co.kr/product/detail.html?product_no=27'
 'http://m.maychic.co.kr/product/detail.html?product_no=32']


In [17]:
'''
문자 요구사항: 이름은 문자나 숫자로 시작해야 합니다.

제한: 75 (영문 150)

유형: String

예: 작업용 부츠 - 사이즈 7.5
'''
tempdf['title'] = df['title'].apply(
    lambda x : ff.setText(x)
                .limitTextLength(75)
                .getText()                
)
print(tempdf['title'][:3])

0                  [메이시크] 아이브로우 가이드 4종 1set #롱래스팅
1                   [메이시크] 쿨톤 벨벳틴트 선착순특가 품절임박 #착붙
2    [메이시크] 핑거팁 타투 아이브로우 틴트+가이드4종 증정 #눈썹타투펜 #롱래스팅
Name: title, dtype: object


In [18]:
'''
문자 요구 사항: 소수점 구분 기호로 마침표(.)를 사용하고 천 단위 쉼표(,)는 사용하지 않습니다.

제한: 14

유형: String

예: 1299.99
'''
tempdf['price'] = df['price'].apply(
    lambda x : ff.setText(x)
                .fitNumber()
                .limitTextLength(14)
                .getText()
)
print(tempdf['price'][:3])

0     3500
1     7900
2    14900
Name: price, dtype: object


In [19]:
'''
문자 요구 사항: 소수점 구분 기호로 마침표(.)를 사용하고 천 단위 쉼표(,)는 사용하지 않습니다.

제한: 50
유형: String

예: 99.99
'''
tempdf['sale_price'] = df['sale_price'].apply(
    lambda x : ff.setText(x)
                .fitNumber()
                .limitTextLength(50)
                .getText()
)
print(tempdf['sale_price'][:3])

0     3500
1     7900
2    14900
Name: sale_price, dtype: object


In [20]:
'''
문자 요구 사항: 값은 8, 12, 13 또는 14 자리 숫자 (UPC, EAN, JAN 또는 ISBN) 여야합니다.

제한: 50

유형: String

예: 0001234560012
 
'''
def getGtin(x):
    ff = FeedFilter('')
    ff.setText(x)
    if ff.checkNumberLengthInList([8,12,13,14]) :
        return ff.getText()
    else : 
        return False
    
tempdf['gtin'] = df['gtin'].apply(
    lambda x : getGtin(x)
)
print(df['gtin'][:3])
print(tempdf['gtin'][:3])

0   NaN
1   NaN
2   NaN
Name: gtin, dtype: float64
0    False
1    False
2    False
Name: gtin, dtype: bool


In [21]:
'''
제한: 70

유형: String

예: 4060882
 
'''

tempdf['mpn'] = df['mpn'].apply(
    lambda x : ff.setText(x)
                .limitTextLength(70)
                .getText()
)
print(tempdf['mpn'][:3])

0    nan
1    nan
2    nan
Name: mpn, dtype: object


In [22]:
'''
제한: 70

유형: String

예: Criteo
'''
tempdf['brand'] = df['brand'].apply(
    lambda x : ff.setText(x).getText()
)
print(tempdf['brand'][:3])

0    메이시크
1    메이시크
2    메이시크
Name: brand, dtype: object


In [23]:
'''
주의할 점: product_type는 문자 또는 숫자로 시작해야 하며 ASCII 문자만 포함할 수 있습니다. 
카테고리들은 >로 구분되어야 합니다 (XML 카탈로그의 경우는 &gt;). 저희는 전체 카테고리 경로만 지원합니다. 
제품이 여러 카테고리로 제공되는 경우, 먼저 기본 카테고리를 시작으로 쉼표 (,)로 다른 카테고리들을 보내주시면 됩니다. 
카테고리 경로에 ID는 허용되지 않습니다. ID는 product_type_key 필드에 입력해야 합니다. 

제한: 500

유형: String

예:

단일: Computing>Keyboards and Mice>Mice>Wireless Mice
다중: Computing>Keyboards and Mice>Mice>Wireless Mice,Gifts>For Dad>Mice
'''

tempdf['product_type'] = df['product_type'].apply(
    lambda x : ff.setText(x)
                .limitTextAscii()
                .limitTextLength(500)
                .getText()
)
print(df['product_type'][:])
print(tempdf['product_type'][:])

0                 ALL 
1                 ALL 
2                 ALL 
3                 ALL 
4                 ALL 
5                 ALL 
6                 ALL 
7                 ALL 
8                 ALL 
9                 ALL 
10                ALL 
11                ALL 
12                ALL 
13                ALL 
14                ALL 
15                ALL 
16                ALL 
17                ALL 
18                ALL 
19                ALL 
20                ALL 
21                ALL 
22                ALL 
23           ★ 최저가 보장 
24                ALL 
25                ALL 
26                ALL 
27                ALL 
28                ALL 
29                ALL 
30                ALL 
31                ALL 
32                ALL 
33                ALL 
34                ALL 
35                ALL 
36                ALL 
37                ALL 
38                ALL 
39                ALL 
40                ALL 
41                ALL 
42                ALL 
43         

In [24]:
tempdf.head()

,id,availability,condition,description,image_link,link,custom_label_0,title,price,sale_price,gtin,mpn,brand,product_type
0,13,in stock,NEW,적립금 있음 | 배송비 2500 | 리뷰 : 58,http://maychic.co.kr/web/product/medium/202008...,http://maychic.co.kr/product/detail.html?produ...,http://m.maychic.co.kr/product/detail.html?pro...,[메이시크] 아이브로우 가이드 4종 1set #롱래스팅,3500,3500,False,nan,메이시크,ALL
1,27,in stock,NEW,적립금 있음 | 배송비 2500 | 리뷰 : 77,http://maychic.co.kr/web/product/medium/202008...,http://maychic.co.kr/product/detail.html?produ...,http://m.maychic.co.kr/product/detail.html?pro...,[메이시크] 쿨톤 벨벳틴트 선착순특가 품절임박 #착붙,7900,7900,False,nan,메이시크,ALL
2,32,in stock,NEW,적립금 있음 | 배송비 2500 | 리뷰 : 1153,http://maychic.co.kr/web/product/medium/201908...,http://maychic.co.kr/product/detail.html?produ...,http://m.maychic.co.kr/product/detail.html?pro...,[메이시크] 핑거팁 타투 아이브로우 틴트+가이드4종 증정 #눈썹타투펜 #롱래스팅,14900,14900,False,nan,메이시크,ALL
3,60,in stock,NEW,적립금 있음 | 배송비 2500 | 리뷰 : 103,http://maychic.co.kr/web/product/medium/202008...,http://maychic.co.kr/product/detail.html?produ...,http://m.maychic.co.kr/product/detail.html?pro...,[메이시크]24시간 유지되는 듀오립 #착붙,9900,9900,False,nan,메이시크,ALL
4,70,in stock,NEW,쿠폰 할인 | 적립금 있음 | 무료배송 | 리뷰 : 1136,http://maychic.co.kr/web/product/medium/202008...,http://maychic.co.kr/product/detail.html?produ...,http://m.maychic.co.kr/product/detail.html?pro...,[메이시크] 메이퀸 빅 커버쿠션 25g (픽서+퍼프 증정!),20400,20400,False,nan,메이시크,ALL


## Export

In [25]:
tempdf.to_csv('./data/export_ep.tsv', index=False, header=True, sep='\t')    